# 在FPGA上部署SG IR

本教程将继续介绍如何最终部署算法。在这里是将面部检测SSD模型部署到FPGA中。我们已经讨论了网络本身，Tensorflow作为**DSL**，我们将通过使用在之前的教程中引入的平台来完成这个网络，并将其实际部署到FPGA上。

教程安排:

1. [冻结模型](#冻结模型)
2. [创建SG](#创建SG)
3. [优化SG](#优化SG)
4. [硬件改进](#硬件改进)
5. [将SG导入raintime](#将SG导入raintime)
6. [编译观察](#编译观察)


### 教程目标:

1. 学习并了解编译过程是如何被分解的
2. 了解如何使用我们的工具流程将模型部署到FPGA上

鉴于在教程一中已安装好了Plumber，我们现在可以用他来生成数据流图（SG）。该图可以在平台上传递，在FPGA和CPU上最终执行。在本实验之后，你可以了解多级编译直达FPGA层次的概念，或者可以给你自己的项目激发一些灵感。

快速的知识总结再现：平台本身由多个部分组成：Plumber是一个基于Web的应用程序，能够对机器学习算法进行模板化描述，优化它并创建SG，然后将其传递到raintime。Raintime实例化计算节点，要么在CPU中处理，要么卸载到FPGA加速器。rainman 然后获取FPGA模板并在设备本身上综合它们，同时与在CPU上实例化的节点互连。 所有过程都可以在一个简单的图表中可视化：

![Flowchart.png](../data/figs/platform_flowchart.png)

为了深入编译过程，可以将其分为两部分。第一部分是软件部分Plumber。平台的第一层得到CNN的描述，然后将其传递到SG IR中，然后它在软件，硬件中进行优化，硬件将生成`* .json`结构，这将成为实际硬件设计的指南。之后，这个描述将被用来在Vivado环境中创建比特流。关于我们平台所有的信息和功能可以在此链接中了解：[link](https://corerain.github.io/plumber-docs/topics/commands.html).

在软件之后，我们需要在下面讨论很多步骤。

首先拿SSD做一个例子，先确保在你的模型中有checkpoint文件并且`plumber_cli`已在你的虚拟环境里。checkpoint文件已经为你下载好。

然后我们可以简单利用`plumber_cli`来一步步创建一个能被上传到FPGA上的SG。您可以在教程的根目录中创建一个单独的终端窗口，或者只是从jupyter notbook的comfort中运行命令。

## 冻结模型

确保你的checkpoint文件在训练或者再训练之后包含以下文件：

- `checkpoint`: 包含有关checkpoint目录的元信息，数据文件和索引文件的文件
- `*.meta`: 模型元信息
- `*.data`: 权重数据
- `*.index`: 索引文件

这些文件现在将被导入**Plumber**，并转换为平台可理解并可以优化的表示。

In [21]:
! plumber_cli freeze ../model/ssd_ckpt -d ../model/ssd -o ssd_KYnet_v2/block9_box/Reshape,ssd_KYnet_v2/block8_box/Reshape,ssd_KYnet_v2/block4_box/Reshape,ssd_KYnet_v2/softmax_5/Reshape_1,ssd_KYnet_v2/softmax_1/Reshape_1,ssd_KYnet_v2/softmax_2/Reshape_1,ssd_KYnet_v2/softmax_3/Reshape_1,ssd_KYnet_v2/block7_box/Reshape,ssd_KYnet_v2/block10_box/Reshape,ssd_KYnet_v2/softmax/Reshape_1,ssd_KYnet_v2/block3_box/Reshape,ssd_KYnet_v2/softmax_4/Reshape_1

TensorFlow Version: 1.5.0
Freezing ../model/ssd_ckpt to ../model/ssd/model.pb ...
Output node names: ['ssd_KYnet_v2/block9_box/Reshape', 'ssd_KYnet_v2/block8_box/Reshape', 'ssd_KYnet_v2/block4_box/Reshape', 'ssd_KYnet_v2/softmax_5/Reshape_1', 'ssd_KYnet_v2/softmax_1/Reshape_1', 'ssd_KYnet_v2/softmax_2/Reshape_1', 'ssd_KYnet_v2/softmax_3/Reshape_1', 'ssd_KYnet_v2/block7_box/Reshape', 'ssd_KYnet_v2/block10_box/Reshape', 'ssd_KYnet_v2/softmax/Reshape_1', 'ssd_KYnet_v2/block3_box/Reshape', 'ssd_KYnet_v2/softmax_4/Reshape_1']
2018-07-24 17:35:28 INFO Initialising with model directory ...
2018-07-24 17:35:28.696265: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2
Converted 52 variables to const ops.
Successfully writen the frozen graph to ../model/ssd/model.pb


- `../model/ssd_ckpt/`: is the checkpoint directory
- `../model/ssd/`: is the output directory

## 创建SG

在已经创建好SG的文件之外，你可以再次使用`plumber_cli`生成一个原始的数据流图. 这个命令实际上是解析了来自冻结的TensorFlow模型的内容并生成相应的SG。如果您希望在运行命令后查看原始.pbtxt请查看`model/ssd/ssd_sg.pbtxt`。

In [22]:
!plumber_cli SG \
    --model-file=../model/ssd/model.pb \
    --SG-bin-file=../model/ssd/model_SG.pb \
    --SG-text-file=../model/ssd/ssd_SG.pbtxt \
    --SG-data-file=../model/ssd/ssd_SG.h5 \
    --input-image-shape=1,256,256,3

TensorFlow Version: 1.5.0
Generating SG from ../model/ssd/model.pb to ../model/ssd/model_SG.pb
2018-07-24 17:35:36 INFO loading TensorFlow model from ../model/ssd/model.pb ...
2018-07-24 17:35:36 INFO Initialising with model frozen file ...
2018-07-24 17:35:36 INFO Successfully loaded the model!
2018-07-24 17:35:36 INFO Rewriting the input TensorFlow Graph for convenient SG generation ...
2018-07-24 17:35:36 INFO Rewriting the graph by "ReshapeRewriter" ...
2018-07-24 17:35:36 INFO Rewriting the graph by "PlaceholderRewriter" ...
2018-07-24 17:35:36 INFO Rewriting the graph by "DropoutRewriter" ...
2018-07-24 17:35:36 INFO Building a TFGraph object ...
2018-07-24 17:35:36 INFO Running model inference, it may take a while ...
2018-07-24 17:35:36.888671: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2
2018-07-24 17:35:36 INFO Running inference ...
2018-07-24 17:35:37 INFO Done!

  device: CPU
  type: T_FLOAT
  conv2d_op_param {
    depth: 8
    kernel_size: 3
    pad: 1
    stride: 1
    activation_fn: ""
    use_maxpool_2x2: false
    use_batch_norm: false
    use_bias: true
    use_relu: false
  }
}
node {
  name: "ssd_KYnet_v2/softmax_3/reshape_1"
  input: "ssd_KYnet_v2/softmax_3/softmax"
  op: "Reshape"
  device: CPU
  type: T_FLOAT
  reshape_op_param {
    shape {
      dim: 384
      dim: 2
    }
  }
}
node {
  name: "ssd_KYnet_v2/softmax_4/reshape"
  input: "ssd_KYnet_v2/block9_box/reshape_1"
  op: "Reshape"
  device: CPU
  type: T_FLOAT
  reshape_op_param {
    shape {
      dim: 1
      dim: 4
      dim: 4
      dim: 6
      dim: 2
    }
  }
}
node {
  name: "ssd_KYnet_v2/softmax_4/softmax"
  input: "ssd_KYnet_v2/softmax_4/reshape"
  op: "Softmax"
  device: CPU
  type: T_FLOAT
  softmax_op_param {
    shape {
      dim: 96
      dim: 2
    }
  }
}
node {
  name: "ssd_KYnet_v2/block10_box/reshape_1"
  input: "ssd_KYnet_v2/block10_box/conv_cls/conv2d"
 

- `../model/ssd/model.pb`: is the Plumber file describing the network as a Plumber binary file
- `../model/ssd/model_sg.pb`: is the Plumber template for a SG
- `../model/ssd/ssd_sg.pbtxt`: is the description of the SG in a text format
- `../model/ssd/ssd_sg.h5`: is a data-file describing input/output sizes, important for random data generation or weights extraction
- `1,256,256,3`: is an input image shape, in our case 256x256 images with three channels with one image per batch, n.b.: the format is Batch Size, Height, Width, Number of Channels.

## 优化SG

SG在硬件部署之前需要被优化，其过程两步：平台无关优化和平台相关优化。平台无关优化改善了模型在不清楚平台规格下的效率，而平台相关优化则将这些信息考虑在内。这两步都可以在`plumber_cli`命令中调用。

### 平台无关优化

`sg_opt` 以平台无关的方式进行输入SG的优化，它主要量化模型系数。

In [23]:
!plumber_cli SG_opt \
    --SG-file=../model/ssd/model_SG.pb \
    --SG-data-file=../model/ssd/ssd_SG.h5 \
    --opt-SG-file=../model/ssd/ssd_opt_SG.pbtxt \
    --logdir=../model/ssd/logs

TensorFlow Version: 1.5.0
Loading SGDef from file "../model/ssd/model_SG.pb" ...
Loading SG from definition and data ...
2018-07-24 17:35:46 INFO Loading data from file ../model/ssd/ssd_SG.h5 ...
Initialise SG optimizer ...
Running optimization ...
2018-07-24 17:35:46 INFO Running optimization pass "Data Representation Optimization" ...
Explored data representation table:
                            node_name         key       min         max rep_fixed32 rep_ufixed32 rep_fixed16 rep_ufixed16 rep_fixed8 rep_ufixed8
0                           img_input       ifmap  0.001562  255.994907       22, 9        23, 9        6, 9         7, 9       None        None
1                           img_input       ofmap  0.001562  255.994907       22, 9        23, 9        6, 9         7, 9       None        None
2         ssd_KYnet_v2/conv1_1/conv2d       ifmap  0.001562  255.994907       22, 9        23, 9        6, 9         7, 9       None        None
3         ssd_KYnet_v2/conv1_1/conv2d       o

All reports are written to: ../model/ssd/logs/reports/SG_opt


This will now take the original SG described in `tmp/ssd_SG.pb` and optimize it to maximaise the gain from our platform: 

- `../model/ssd/model_SG.pb`: is the previously generated SG file
- `../model/ssd/ssd_SG.h5`: is the data file that we have created in the previous step
- `../model/ssd/ssd_opt_SG.pbtxt`: this is the new, optimised, pbtxt
- `../model/ssd/logs`: this is the logging directory

现在让我们进入下一步，一个真正在FPGA的嵌入式系统执行的过程。

### 平台相关优化

`hdl_opt` 命令执行平台相关优化。它将板规格文件和平台无关优化的SG作为输入，并生成进一步优化的SG和指定硬件设计参数的配置文件

In [25]:
! plumber_cli hdl_opt \
    ../data/boards/rainman_board_v2.pbtxt \
    --SG_file=../model/ssd/ssd_opt_SG.pbtxt \
    --SG_data_file=../model/ssd/ssd_SG.h5 \
    --opt-SG-file=../model/ssd/ssd_hdl_SG.pbtxt \
    --logdir=../model/ssd/logs

TensorFlow Version: 1.5.0
Optimise SG for hardware-accelerated execution ...
LOAD BoardDef from file: "../data/boards/rainman_board_v2.pbtxt"
2018-07-24 17:40:02 INFO Loading data from file ../model/ssd/ssd_SG.h5 ...
LOAD SG from SGDef file: "../model/ssd/ssd_opt_SG.pbtxt" and data file: "../model/ssd/ssd_SG.h5"
RUN  HDL optimization ...
2018-07-24 17:40:02 INFO Running optimization pass "Device Optimization" ...
2018-07-24 17:40:02 INFO Marked node "img_input" with "device: FPGA"
2018-07-24 17:40:02 INFO Marked node "ssd_KYnet_v2/conv1_1/conv2d" with "device: FPGA"
2018-07-24 17:40:02 INFO Marked node "ssd_KYnet_v2/conv2_1/conv2d" with "device: FPGA"
2018-07-24 17:40:02 INFO Marked node "ssd_KYnet_v2/conv2/conv2_1/conv2d" with "device: FPGA"
2018-07-24 17:40:02 INFO Marked node "ssd_KYnet_v2/conv2/conv2_2/conv2d" with "device: FPGA"
2018-07-24 17:40:02 INFO Marked node "ssd_KYnet_v2/conv3/conv3_1/conv2d" with "device: FPGA"
2018-07-24 17:40:02 INFO Marked node "ssd_KYnet_v2/conv3/conv

在以上的案例中，`data/boards/rainman_board_v2.pbtxt`是一个`Rainman V2`加速卡专用的板卡专用文件。它仅描述在我们目标板上每一种类型资源的数量。

```protobuf
name: "RAINMAN_BOARD_V2"
num_lut: 218600
num_ff: 437200
num_bram: 545
num_dsp: 900
```

这个优化过程将尝试融合运算符，为每个运算分配执行设备，并探索设计空间。你可以通过详细输出检查整个过程。`../model/ssd/ssd_hdl_sg.pbtxt`是优化的SG，而在 `../model/ssd/logs`中你可以找到一个`hdl_params.json`文件，包含了硬件设计参数。这个JSON文件将进一步被用来生成比特流。

## 硬件部署

为了创立比特流，我们需要提供`hdl_params.json`给`gen`命令。`gen`命令会调用FPGA综合工具链生成比特流。因为我们在虚拟机或JupyterHub中没有包含任何FPGA工具，所以我们只演示命令和相关输出。

## 上传SG到raintime

快速回顾: `raintime`是一个用于在嵌入式FPGA系统上处理CNN的软件运行时库。CNN中的计算节点既可以在CPU中处理，也可以卸载到由`rainman`构建的FPGA加速器设计中。 它还有几个部分，可以在图表中进行总结，而不需要过多细节：

![raintime.png](../data/figs/raintime.png)

节点自身已经在raintime上实现，但是我们需要但我们需要简化执行并告知我们想要提取的数据等。在以后的版本中，这一步将是完全自动的，目前我们必须编写一个简短的演示

如果我们要写一个演示在`raintime`，并在FPGA上执行这一演示，还需要以下步骤：

```C++
  int batch_size = 1;
  int n_channels = 3;
  int img_size = 265

  // Load image
  cv::Mat image;
    image = cv::imread(argv[1], cv::CV_LOAD_IMAGE_COLOR);
    
  // Reorder the pixel values from the default ordering of opencv
  std::vector<uint8_t> converted;
  auto image_pointer = image.ptr();
  for (size_t i = 0; i < n_channels; i++) {
    for (size_t j = 0; j < img_size * img_size; j++) {
      converted[j + img_size * img_size * i] =  image[n_channels * j + i];
    }
  }
  
  // Load SGDef
  auto SG_def = LoadSGDefFromFile(SG_file_name);

  // Use the integrated builder to build the graph and make abstractions to connect the CPU and FPGA
  *SG = SGBuilder(SG_def).Build();

  // Load constant data map, including weights and biases
  *data_map = new SGDataMap;
  (*data_map)->LoadFromDir(data_dir);
  
  
  std::vector<int> dims;
  dims.push_back(n_channels);
  dims.push_back(img_size);
  dims.push_back(img_size);
  dims.push_back(batch_size);
  
  
  // Load input data-map into the SG, without any particular pre-processing optimisations
  SGDataMap *input_data_map = new SGDataMap;
    input_data_map->LoadImage(converted, image_size, n_channels,
                              "input_tensor", dims, "no");
                              
  // Extract the output data map from the runner, in this case the 
  auto output_data_map = runner->Run(SG, data_map, input_data_map, true);
  auto output_data = output_data_map->get("Predictions").second;


  std::cout<<"The number of detected faces: "<<output_data_map.size()<<std::endl;
  
  //House-keeping
  delete input_data_map;
  delete output_data_map;
```

Once you have finished writing the demo, then you would have to compile your design, on the board.

```bash
$ git clone https://github.com/corerain/raintime.git
$ cd raintime
$ mkdir build && cd build
# Create the compiling structure through CMake and specify the number of fraction bits (FB) for a 32 bit representation
$ cmake .. -DCMAKE_BUILD_TYPE=Release -DDEF_FIXED_NUM_FB_32=20 -DBUILD_TESTS=ON
$ make
$ ./your_demo
```

### 编译观察

这一部分很简单，`raintime`有几个设置关于如何编译项目，在这里我们会尽量避免很多细节问题。一旦连接到具有预安装的OS和正确的`BOOT.bin`板卡上，你可以在你的演示中克隆raintime的项目然后编译它。

... 到现在，你已经在Python / Tensorflow中写入了一个算法并在FPGA上执行它，是不是很棒？

#### 资源

还记得上一个教程里的定点表示吗？除了准确性和执行速度意外，它显然也会影响所使用的资源。让我们看看每种数据表示中各使用了多少资源（16位和32位）

| SSD Demo          | 32-bits | 16-bits |
|-------------------|---------|---------|
| Registers         | 280871  | 92780   |
| Block Memory bits | 12Mb    | 6349Kb  |
| DSP Blocks        | 168     | 150     |

你可以看到寄存器的数量大约减少了三倍，并且BRAM的使用量减少了大约两倍。

raintime的运行流程也可以在网上了解： [Link](http://ai-dev.corerain.com:8000/)，在该网址上你可以查看包括SSD在内的更多demo。下图是鲲云科技团队。

![team.jpeg](../data/figs/team.jpeg)

或者你可以观看视频演示:

In [1]:
%%HTML
<video width="640" height="480" controls>
  <source src="../data/figs/face_detecion.mp4" type="video/mp4">
</video>

### 联系方式

如果你想了解更多信息，请随时与我们联系:

-   Professor Wayne Luk (w.luk@imperial.ac.uk)

-   Martin Ferianc (martin.ferianc@corerain.com)

-   Ruizhe (Vincent) Zhao (vincent.zhao@corerain.com)